<a href="https://colab.research.google.com/github/anandhc6/Assignment1/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

In [ ]:
import numpy as np
from numpy import random
from keras.datasets import fashion_mnist
#import the required libraries
from matplotlib import pyplot
import math as math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

(X, y), (X_test, y_test) = fashion_mnist.load_data()

# Reshaping the data 
X = X.reshape(X.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)

# Normalizing
X = X/255.0
X_test = X_test/255.0

# Split the data X into a training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

#Layer dimensions
la =[784,256,128,10]

L=len(la)-1

#weights and bias empty initializing
np.random.seed(42)
params={}

In [ ]:
def weight_initialize(weight_para):
    if weight_para == 'Xavier':
        for i in range(0,len(la)-1):
            params["W"+str(i)]=np.random.randn(la[i+1],la[i])*np.sqrt(2/la[i+1])
            params["b"+str(i)] = np.zeros((la[i+1], 1))
    if weight_para == 'random':
        for i in range(0,len(la)-1):
            params["W"+str(i)] = np.random.randn(la[i+1], la[i]) * 0.01
            params["b"+str(i)] = np.zeros((la[i+1], 1))

In [ ]:
#Activation function     
def sigmoid(x):
     return 1. /(1. + np.exp(-x))

def activation_function(x,activation):
    if activation=='sigmoid':
        return (1.0 / (1.0 + np.exp(-x)))
    elif activation=='sigmoid_derivative':
        return (activation_function(x,'sigmoid') * (1 - activation_function(x,'sigmoid')))
    elif activation=='relu':
        return x * (x > 0)
    elif activation=='relu_derivative':
        x[x <= 0.0] = 0.0
        x[x > 0.0] = 1.0
        return x
    elif activation=='tanh':
        return np.tanh(x)
    elif activation=='tanh_derivative':
        return 1 - (activation_function(x,'tanh') ** 2)
    else:
        raise Exception("Invalid activation function",activation)
def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft


In [ ]:
def feedforward(X, params,activation_para):

    L = len(params)//2  
    A = [None]*L # activations
    H = [None]*L # pre-activations
     
    k=0
    A[0]=np.dot(params["W"+str(k)],X)+params["b"+str(k)]
    H[0]=activation_function(A[k],activation_para)
   
    for k in range(1,L-1):
            A[k]=np.dot(params["W"+str(k)],H[k-1])+params["b"+str(k)]
            H[k]=activation_function(A[k],activation_para)
   
    k=L-1
    A[k]=np.dot(params["W"+str(k)],H[k-1])+params["b"+str(k)]
    H[k]=softmax(A[k])
   
    y=H[-1]
    return A,H,y



In [ ]:

def back_propagation(X_train,y_train,params,A ,H,activation_para,loss):
    #Initialzing gradients
    L = len(params)//2
    #A = [None]*L # activations
    #H = [None]*L
    g_weights = [0]*L
    g_biases  = [0]*L
    g_a       = [0]*(L+1)
    g_h       = [0]*(L+1)
    n_samples = X_train.shape[0]  # Change depending on the dimensions of data
   
    # print(str(activation_para)+'_derivative')

    for k in range(L-1,-1,-1): #2,1,0
       
        if k == L-1:
            if loss == 'cross_entropy':
                g_a[k] = H[k]  - y_train  # keep or remove T depending on the dimensions of data
            elif loss == 'square_loss':
                g_a[k] = (H[k] - y_train) * H[k] * (1 - H[k])
           
       
        else:
            g_h[k] = (1/n_samples)*np.dot(params["W"+str(k+1)].T,g_a[k+1])
            if activation_para == 'sigmoid':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],activation_function(A[k], str(activation_para)+'_derivative'))
            elif activation_para == 'tanh':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],activation_function(A[k], str(activation_para)+'_derivative'))
            elif  activation_para == 'relu':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],activation_function(A[k], str(activation_para)+'_derivative'))


        if k == 0:
            g_weights[k] = (1/n_samples)*np.dot(g_a[k],X_train.T)
        else:
            g_weights[k] = (1/n_samples)*np.dot(g_a[k],H[k-1].T)

        g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
    return g_weights,g_biases

In [ ]:
def loss_accuracy(x,params,y,t,loss,activation_para, n_class=10,n_examples = 54000):
    _,_,yhate=feedforward(x, params,activation_para)
    print(x.shape)
    print(y.shape)
    # ls=[]
    # accuracy=[]
    if loss == 'cross_entropy':
        one_hot = np.zeros((n_class, x.shape[1]))
        one_hot[y, np.array(list(range(x.shape[1])))] = 1
        ls= -np.sum(one_hot *np.log(yhate))/x.shape[1]
                  
    if loss == 'squared_loss':
        eIndicator = np.zeros((n_class, n_examples))
        eIndicator[y, np.arange(n_examples)] = 1
        ls =np.sum((yhate - eIndicator)**2) / n_examples
    
    yh=np.argmax(yhate,axis=0)
    accuracy = accuracy_score(y, yh)*100 
     
    # for p in range(len(ls)):
    print("loss of " +str(t) +" = " + str(ls) )
    print("Accuracy of " +str(t) +" = " + str(accuracy) )
    return accuracy,ls

In [ ]:
def vanillagd(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para):
    t=0
    A=[]
    H=[]
    while t<epochs:
        for i in range(0,minibatch,1):
                    X_mini=x_train[(i*batch):((i+1)*batch)]
                    y_mini=y_train[(i*batch):((i+1)*batch)]
                    batch1 = X_mini.shape[0]
       
                    y_mini_one_hot = np.zeros((10, batch1))
                    y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
                    A,H,y_hat= feedforward(X_mini.T,params,activation_para)
                    g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)
                    for j in range(0,L):
                      g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                      params['W'+str(j)]-=eta*g_weights[j]
                      params['b'+str(j)]-=eta*g_biases[j]
                   
        t+=1
   
        train_acc,train_loss = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
       
        val_acc,val_loss = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        return train_acc,train_loss,val_acc,val_loss
       

In [ ]:
# MOMENTUM_GD optimizer 
        
def momentumGD(x_train,y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay):
    preW = {}
    preb = {}
    gamma = 0.9
   
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
    t=0
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=x_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params)
            g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,params,A ,H)
            for j in range(0,L):
                g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                w=(gamma*preW['W'+str(j)])+eta*g_weights[j]
                b=(gamma*preb['b'+str(j)])+eta*g_biases[j]
                params['W'+str(j)]-=w
                params['b'+str(j)]-=b
                preW['W'+str(j)]=w
                preb['b'+str(j)]=b
        t+=1
        
        #Loss & accuracy
        print('completed feed and back per epoch')
        loss_accuracy(x_train.T,params,y_train,t,loss)


In [ ]:
#NAG optimizer  
       
def nag(x_train,y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay):
    t=0
    # batch=600
    gamma = 0.9
    preW={}
    preb={}
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
        # print(preW['W'+str(k)].shape)
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=x_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params)
            g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,params,A ,H)
            l_params={}
            for j in range(0,L):
                l_params['W'+str(j)]=params['W'+str(j)]-(gamma*preW['W'+str(j)])
                l_params['b'+str(j)]=params['b'+str(j)]-(gamma*preb['b'+str(j)])
            for j in range(0,L):
                g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,l_params,A ,H)
                g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                w=(gamma*preW['W'+str(j)])+eta*g_weights[j]
                b=(gamma*preb['b'+str(j)])+eta*g_biases[j]
                params['W'+str(j)]-=w
                params['b'+str(j)]-=b
                preW['W'+str(j)]=w
                preb['b'+str(j)]=b
        t+=1
       
        #Loss & accuracy            
        print('completed feed and back per epoch')
        loss_accuracy(x_train.T,params,y_train,t,loss)
       

In [ ]:
#RMSprop optimizer
        
def rms(x_train,y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay):
    t=0
    gamma = 0.9
    eps = 1e-8
    beta = 0.9
    preW={}
    preb={}
    
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=X_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params)
            g_weights,g_biases=back_propagation(X_mini.T,y_mini_one_hot,params,A ,H)  
            for j in range(0,L):
                g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                w=(beta*preW['W'+str(j)])+(1.0-beta)*(g_weights[j])**2
                b=(beta*preb['b'+str(j)])+(1.0-beta)*(g_biases[j])**2
                params['W'+str(j)]-=((eta/(np.sqrt(w+eps)))*g_weights[j])
                params['b'+str(j)]-=((eta/(np.sqrt(b+eps)))*g_biases[j])
                preW['W'+str(j)]=w
                preb['b'+str(j)]=b
        t+=1
        
        #Loss & accuracy
        print('completed feed and back per epoch')
        loss_accuracy(x_train.T,params,y_train,t,loss)


In [ ]:
#STOCHASTIC optimizer
       
def sgd(x_train,y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay):
    t=0
    while t<epochs:
        indexes=np.random.permutation(len(x_train))
        X_ran=x_train[indexes]
        Y_ran=y_train[indexes]
        for x,y in zip(X_ran,Y_ran):
            X_mini=np.array([x])
            y_mini=np.array([y])
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params)
            g_weights,g_biases=back_propagation(X_mini.T,y_mini_one_hot,params,A ,H)
            for j in range(0,L):
                g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                params['W'+str(j)]-=eta*g_weights[j]
                params['b'+str(j)]-=eta*g_biases[j]
        t+=1
       
        #Loss & accuracy
        print('completed feed and back per epoch')
        loss_accuracy(x_train.T,params,y_train,t,loss)
     